In [67]:
import numpy as np
import math

class Node:
    def __init__(self, nodec, gaussrad, const):
        self.Coordinates = nodec
        self.GausRadius = gaussrad
        self.GausConstant = const

        
Nodes = []
"""
Nodes.append(Node(np.array([0,0]), 4, 16))
Nodes.append(Node(np.array([0.5,0]), 4, 16))
Nodes.append(Node(np.array([0,0.5]), 4, 16))
Nodes.append(Node(np.array([1,0]), 4, 16))
Nodes.append(Node(np.array([0,1]), 4, 16))
Nodes.append(Node(np.array([0.5,0.5]), 4, 16))
Nodes.append(Node(np.array([0.5,1]), 4, 16))
Nodes.append(Node(np.array([1,0.5]), 4, 16))
Nodes.append(Node(np.array([1,1]), 4, 16))
"""
Nodes.append(Node(np.array([0,0]), 4, 16))
Nodes.append(Node(np.array([0.3,0]), 4, 16))
Nodes.append(Node(np.array([0,0.2]), 4, 16))
Nodes.append(Node(np.array([1.2,1.4]), 4, 16))
Nodes.append(Node(np.array([0.4,1.2]), 4, 16))
Nodes.append(Node(np.array([0.8,0.9]), 4, 16))
Nodes.append(Node(np.array([0.9,1.0]), 4, 16))
Nodes.append(Node(np.array([1.3,0.5]), 4, 16))
Nodes.append(Node(np.array([1.2,1.1]), 4, 16))
Nodes.append(Node(np.array([2.0,1.8]), 4, 16))
Nodes.append(Node(np.array([0.8,2.0]), 4, 16))
Nodes.append(Node(np.array([1.6,1.9]), 4, 16))

#Gaussian weight function
def GauseWeight(X, Coordinates, GausRadius, GausConstant):
    dist = np.linalg.norm(X-Coordinates)
    weight = (math.exp(-(dist/GausConstant)**2) - 
        math.exp(-(GausRadius/GausConstant)**2))/(1 - math.exp(-(GausRadius/GausConstant)**2))
    if dist < GausRadius:
        return weight
    else:
        return 0

def Weight(Node1, Node2):
    return GauseWeight(Node2.Coordinates, Node1.Coordinates, Node1.GausRadius, Node1.GausConstant)

#MLS basis
#linear basis here
BasisSize = 3
DerivEpsilon = 10**(-8)
IntegrateRadius = 10**(-5)

def BasisFunc(i, x):
    if i == 0:
        return 1
    return x[i-1]
def ComputeBasisFunc(X):
    p = []
    for size in range(BasisSize):
        p.append(BasisFunc(size, X))
    return p

def ComputeMatriceP(SomeNodes):
    P = []
    for Node in SomeNodes:
        P.append(ComputeBasisFunc(Node.Coordinates))
    return P
def ComputeMatriceW(X, SomeNodes):
    W = []
    Zeros = np.zeros((len(SomeNodes)))
    index = 0
    for Node in SomeNodes:
        Zeros[index] = GauseWeight(X, Node.Coordinates, Node.GausRadius, Node.GausConstant)
        W.append(Zeros.copy())
        Zeros[index] = 0
        index += 1
    return np.array(W)
def ComputeMatriceA(X, SomeNodes):
    p = np.array(ComputeBasisFunc(X))  
    P = np.array(ComputeMatriceP(SomeNodes))
    W = np.array(ComputeMatriceW(X, SomeNodes))
    return np.array(np.dot(P.T,np.dot(W,P)))
def ComputeMatriceB(X, SomeNodes):
    p = np.array(ComputeBasisFunc(X))  
    P = np.array(ComputeMatriceP(SomeNodes))
    W = np.array(ComputeMatriceW(X, SomeNodes))
    return np.array(np.dot(P.T,W))

def GetDerivp(k):
    #for linear Basis p_{i,k} = [0 ... 1(k) ... 0]
    Deriv = np.array(np.zeros((BasisSize)))
    Deriv[k] = 1
    return Deriv 
def GetDerivMatrice(k, X, MFunction, SomeNodes): # X - only coordinates
    Y = X 
    index = 0
    for y in Y:
        if index == k:
            y += DerivEpsilon
            break
        index += 1
    return (MFunction(Y, SomeNodes) - MFunction(X, SomeNodes))/DerivEpsilon

def Fi(X, SomeNodes): #X - can be both Node and Node.Coordinates
    p = np.array(ComputeBasisFunc(X))        
    A = ComputeMatriceA(X, SomeNodes) 
    B = ComputeMatriceB(X, SomeNodes)
    return np.dot(p, np.dot(np.linalg.inv(A), B)) #столбец функций Ф.T[i]

def DerivFiAll(X, SomeNodes):
    p = np.array(ComputeBasisFunc(X))
    A = ComputeMatriceA(X, SomeNodes)
    B = ComputeMatriceB(X, SomeNodes)
    Derivs = []
    for k in range(BasisSize):
        Derivp = GetDerivp(k)
        DerivA = GetDerivMatrice(k, X, ComputeMatriceA, SomeNodes)
        DerivB = GetDerivMatrice(k, X, ComputeMatriceB, SomeNodes)
        Derivs.append(np.dot(Derivp.T,np.dot(np.linalg.inv(A), B)) + np.dot(p.T,((-1)*np.dot(np.linalg.inv(A), DerivB) +
            np.dot(np.dot(np.linalg.inv(A), np.dot(DerivA,np.linalg.inv(A))), B))))
    return Derivs

def DerivFi(k, X, SomeNodes):
    p = np.array(ComputeBasisFunc(X))
    A = ComputeMatriceA(X, SomeNodes)
    B = ComputeMatriceB(X, SomeNodes)
    Derivs = []
    Derivp = GetDerivp(k)
    DerivA = GetDerivMatrice(k, X, ComputeMatriceA, SomeNodes)
    DerivB = GetDerivMatrice(k, X, ComputeMatriceB, SomeNodes)
    return np.dot(Derivp.T,np.dot(np.linalg.inv(A), B)) + np.dot(p.T,((-1)*np.dot(np.linalg.inv(A), DerivB) +
            np.dot(np.dot(np.linalg.inv(A), np.dot(DerivA,np.linalg.inv(A))), B)))

def Nu(X, Center):
    return GauseWeight(X, Center.Coordinates, IntegrateRadius, Center.GausConstant)
def DerivNu(k, X, Center):
    Y = X 
    index = 0
    for y in Y:
        if index == k:
            y += DerivEpsilon
            break
        index += 1
    return (Nu(Y,Center)-Nu(X,Center))/DerivEpsilon

for NodeMain in Nodes:
    NodesNear = [] #determine the nodes located in the domain of defnition of the MLS
    for Node in Nodes:
        if Weight(NodeMain, Node) > 0:
            NodesNear.append(Node)
    #MLS approximation
    ShapeFuncsValues = [] #really needed ?
    DerivShapeFuncsValues = [] #?
    for Neighbour in NodesNear:
        ShapeFuncsValues.append(Fi(Neighbour.Coordinates, NodesNear))
        DerivShapeFuncsValues.append(DerivFiAll(Neighbour.Coordinates, NodesNear)) 
        #local Domain - sphere or circle with Radius:
        #weight function - gause weight function with the other GausRadius=IntegrateRadius ?
        #Nu - weight function
        #DerivNu -weight function derivative

